## Data cleaning and preparation

In [2]:
import sys
import pandas as pd
import numpy as np

Durante el curso de análisis y modelado de datos, se dedica una cantidad significativa
de tiempo a la preparación de datos: carga, limpieza, transformación y reorganización.
A menudo se informa que tales tareas ocupan el 80% o más del tiempo de un analista.
A veces, la forma en que los datos se almacenan en archivos o bases de datos no están
en el formato correcto para una tarea en particular. Muchos invetigadores optan por
realizar un procesamiento _ad hoc_ de una forma a otra utilizando un lenguage de
programación de propósito general, como Python, Perl, R o Java, o herramientas de
procesamiento de texto Unix como `sed` o `awk`. Afortunadamente, Pandas, junto con las
características integradas de Python, le proporcionan un conjunto de herramientas de
alto nivel, flexibles y rápidas para permitirle manipular los datos en la forma correcta.

### Manejo de datos faltantes
Los datos faltantes ocurren comúnmente en muchas aplicaciones de análisis de datos.
Uno de los objetivos de Pandas es hacer que trabajar con datos faltantes sea lo más
indoloro posible. Por ejemplo, todas las estadísticas descriptivas sobre objetos
Pandas excluye los datos faltantes de forma predeterminada.

La forma en que los datos faltantes se representan en los objetos Pandas es algo
imperfecta, pero es suficiente para la mayoría del uso del mundo real. Para datos
con dtype `float64`, pandas utiliza el valor de punto flotante `NaN` (no es un número)
para representar los datos faltantes.

Llamamos a esto un valor centinela: cuando está presente, indica un valor faltante (o nulo):

In [3]:
float_data = pd.Series([1.2, -3.5, np.nan, 0])
float_data

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

In [4]:
# isna devuelve una serie booleana con True en valores nulos
float_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

En Pandas, hemos adaptado una convención utilizada en el lenguaje de programación R
refiréndonos a los datos faltantes como NA, que significa _no disponible_. En las
aplicaciones estadísticas, los datos de NA pueden ser datos que no existen o que existen,
pero no se observaron. Al limpiar los datos para el análisis, a menudos es importante
hacer análisis sobre los datos faltantes para identificar problemas de recopilación de
datos o posibles sesgos en los datos causados por los datos faltantes.

In [5]:
# el valor None también se trata como NA
string_data = pd.Series(["aardvark", np.nan, None, "aguacate"])
string_data

0    aardvark
1         NaN
2        None
3    aguacate
dtype: object

In [6]:
string_data.isna()

0    False
1     True
2     True
3    False
dtype: bool

In [7]:
float_data = pd.Series([1, 2, None], dtype='float64')
float_data

0    1.0
1    2.0
2    NaN
dtype: float64

In [8]:
float_data.isna()

0    False
1    False
2     True
dtype: bool

In [9]:
float_data.dropna() # evita los valores NA

0    1.0
1    2.0
dtype: float64

In [10]:
float_data.fillna("NULL") # sustituye valores faltantes por otro valor

0     1.0
1     2.0
2    NULL
dtype: object

In [11]:
float_data.notna() # True en los no faltantes

0     True
1     True
2    False
dtype: bool

### Filtrar datos faltantes
Hay algunas formas de filtrar los datos faltantes. Si bien siempre tiene la opción de
hacerlo a mano usando `pd.isna` e indexación booleana, `dropna` puede ser útil.
En una serie, devuelve la serie con solo los datos no nulos y los valores de índice:

In [12]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [13]:
# es lo mismo que hacer
data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

Con los objetos DataFrame, hay diferentes formas de eliminar los datos faltantes. Es
posible que desee soltar filas o columnas que son todas NA, o solo aquellas filas o
columnas que contienen NA, `dropna` por defecto, suelta cualquier fila que contenga
un valor faltante:

In [14]:
data = pd.DataFrame([[1., 6.5, 3.],
                    [1., np.nan, np.nan],
                    [np.nan, np.nan, np.nan],
                    [np.nan, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [15]:
data.dropna()

,0,1,2
0,1.0,6.5,3.0


Pasando `how="all"` solo soltará filas que sean todas NA:

In [16]:
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


Para soltar columnas `axis="columns"`:

In [17]:
data[4] = np.nan
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [18]:
data.dropna(axis="columns", how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Supongamos que desea mantener solo filas que contengan como máximo un cierto número de
observaciones faltantes. Puede indicar esto con el argumento `thresh`:

In [19]:
df = pd.DataFrame(np.random.standard_normal((7, 3)))
df

,0,1,2
0,1.466794,0.495853,1.188504
1,1.141141,0.388064,-0.075462
2,-0.394911,-0.690867,0.065243
3,-0.149772,0.389615,0.891632
4,-1.797325,0.310991,0.392231
5,0.463541,-1.344578,-0.198229
6,-0.266659,1.400961,1.733136


In [20]:
df.iloc[:4, 1] = np.nan

In [21]:
df

,0,1,2
0,1.466794,NaN,1.188504
1,1.141141,NaN,-0.075462
2,-0.394911,NaN,0.065243
3,-0.149772,NaN,0.891632
4,-1.797325,0.310991,0.392231
5,0.463541,-1.344578,-0.198229
6,-0.266659,1.400961,1.733136


In [22]:
df.iloc[:2, 2] = np.nan
df

,0,1,2
0,1.466794,NaN,NaN
1,1.141141,NaN,NaN
2,-0.394911,NaN,0.065243
3,-0.149772,NaN,0.891632
4,-1.797325,0.310991,0.392231
5,0.463541,-1.344578,-0.198229
6,-0.266659,1.400961,1.733136


In [23]:
df.dropna()

,0,1,2
4,-1.797325,0.310991,0.392231
5,0.463541,-1.344578,-0.198229
6,-0.266659,1.400961,1.733136


In [24]:
df.dropna(thresh=2)

,0,1,2
2,-0.394911,NaN,0.065243
3,-0.149772,NaN,0.891632
4,-1.797325,0.310991,0.392231
5,0.463541,-1.344578,-0.198229
6,-0.266659,1.400961,1.733136


### Rellenando datos faltentes
En lugar de filtrar los datos faltantes (y potencialmente descartar otros datos junto
con ellos), es posible que desee completar los faltantes de varias maneras. Para la
mayoría de los propósitos, el método `fillna` puede ser apropiado. Llamando a `fillna`
con una constante reemplaza los valores faltantes con ese valor:

In [25]:
df.fillna(0)

,0,1,2
0,1.466794,0.000000,0.000000
1,1.141141,0.000000,0.000000
2,-0.394911,0.000000,0.065243
3,-0.149772,0.000000,0.891632
4,-1.797325,0.310991,0.392231
5,0.463541,-1.344578,-0.198229
6,-0.266659,1.400961,1.733136


Llamando `fillna` con un diccionario puede usar un valor diferentes para cada columna:

In [26]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,1.466794,0.500000,0.000000
1,1.141141,0.500000,0.000000
2,-0.394911,0.500000,0.065243
3,-0.149772,0.500000,0.891632
4,-1.797325,0.310991,0.392231
5,0.463541,-1.344578,-0.198229
6,-0.266659,1.400961,1.733136


In [27]:
# cambiar las columnas ya definidas 
# df.columns = ["one", "two", "three"]
# df.index

Los mismo métodos de interpolación disponibles para la reindexación se pueden utiliza
con `fillna`:

In [28]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df

,0,1,2
0,-0.795024,-0.262226,0.289086
1,1.017706,1.659769,-0.910170
2,1.020671,NaN,0.306865
3,-0.126247,NaN,-1.822974
4,-1.398140,NaN,NaN
5,-2.103826,NaN,NaN


In [29]:
df.fillna(method="ffill")

,0,1,2
0,-0.795024,-0.262226,0.289086
1,1.017706,1.659769,-0.910170
2,1.020671,1.659769,0.306865
3,-0.126247,1.659769,-1.822974
4,-1.398140,1.659769,-1.822974
5,-2.103826,1.659769,-1.822974


In [30]:
df.fillna(method="ffill", limit=2)

,0,1,2
0,-0.795024,-0.262226,0.289086
1,1.017706,1.659769,-0.910170
2,1.020671,1.659769,0.306865
3,-0.126247,1.659769,-1.822974
4,-1.398140,NaN,-1.822974
5,-2.103826,NaN,-1.822974


Con `fillna` puede hacer muchaas otras coas, como la simple imputación de datos
utilizando las estadísticas medianas o medias:

In [31]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

### Transformación de datos
El filtrado, la limpieza y otras transformaciones son otra clase de operaciones
importantes.

### Eliminar duplicados
Las filas duplicadas se pueden encontrar en un DataFrame por cualquier número de razones:

In [32]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                    "k2": [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


El método DataFame `duplicated` devuelve una Serie booleana que indica si cada fila es
un duplicado o no:

In [33]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

`drop_duplicated` devuelve un DataFrame con filas donde `duplicated` es `False`:

In [34]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Ambos métodos, por defecto, consideran todas las columnas; alternativamente, puede
especificar cualquier subconjunto de ellas para detectar duplicados. Supongamos que
teníamos una columna adicional de valores y quisíeramos filtrar duplicados basados
solo en la columna `"k1"`:

In [35]:
data["v1"] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [36]:
data.drop_duplicates(subset=["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


`duplicated` y `drop_duplicates` por defecto, mantienen la primera combinación de
valores. Pasando `keep="last"` devolverá el último:

In [37]:
data.drop_duplicates(["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Tranformar datos usando una función o mapeo
Para muchos conjuntos de datos, es posible que desee realizar alguna transformación
basada en valores de una matriz, Serie o columna en un DataFrame.

In [38]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
                             "pastrami", "corned beef", "bacon",
                             "pastrami", "honey ham", "nova lox"],
                    "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Supongamos que desea agregar una columna que indique el tipo de animal del que proviene
cada alimento. Escribamos un mapeo de cada tipo de carne distinto al tipo de animal:

In [39]:
meat_to_animal = {
    "bacon": "pig",
    "pulled pork": "pig",
    "pastrami": "cow",
    "corned beef": "cow",
    "honey ham": "pig",
    "nova lox": "salom"
}

El método `map` en una serie acepta una función o un objeto similar a un diccionario
que contiene un mapeo para hacer la transformación de valores:

In [40]:
data["animal"] = data["food"].map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salom


In [41]:
# usar una función
def get_animal(x):
    return meat_to_animal[x]


data["food"].map(get_animal)

0      pig
1      pig
2      pig
3      cow
4      cow
5      pig
6      cow
7      pig
8    salom
Name: food, dtype: object

In [42]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salom


Usando `map` es una forma conveniente de realizar transformaciones en elementos y otras
operaciones realacionadas con la limpieza de datos.

### Sustitución de valores
Rellenar los datos faltantes con el método `fillna` es un caso especial de reemplazo de
valores más general. `map` se puede utilizar para modificar un subconjunto de valores en
un objeto, pero `replace` proporciona una forma más simple y flexible de hacerlo:

In [43]:
data = pd.Series([1., -999., 2, -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

Los valores `-999` pueden ser valores centienla para datos faltantes. Para reemplazarlos
con valores NA que Pandas entienda, podemos usar `replace` para producir una nueva serie:

In [44]:
data.replace(-999, np.nan, inplace=True)
data

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [45]:
# pasar una lista para varios valores
data.replace([-999, -1000], np.nan, inplace=True)
data

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [46]:
# pasar una lista de sustitución para cada valor
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [47]:
# pasar un diccionario
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [48]:
# pasar un diccionario. usar inplace=True para afectar al DataFrame
data.replace({-999: np.nan, -1000: 0}, inplace=True)
data

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

### Renombar los índices de eje
Al igual que los valores en una serie, las etiquetas de los ejes pueden transformar de
manera similar mediante una función o mapeo de alguna forma para producir objetos
nuevos etiquetados de manera diferente. También puede modificar los ejes en su lugar
sin crear una nueva estructura de datos:

In [49]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                   index=["Ohio", "Colorado", "New York"],
                   columns=["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


al igual que una serie, los índices del eje tienen un método `map`:

In [50]:
def transform(x):
    return x[:4].upper()

data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

Puede asignar el atributo a `index`, modificando el DataFrame en su lugar:

In [51]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


Si desea crear una nueva versión transformada de un conjunto de datos sin modificar el
original, un método útil es `rename`:

In [52]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


En particular, `rename` se puede utilizar junto con un objeto similar a un diccionario,
proporcionando nuevos valores para un subconjunto de las etiquetas de eje:

In [53]:
data.rename(index={"Ohio": "INDIANA"},
            columns={"three": "peekaboo"})

,one,two,peekaboo,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


`rename` le ahorra la tarea de copiar el DataFrame manualmente y asignar nuevos valores
a sus atributos `index` y `columns`.

### Descretization y Binning
Los datos continuos a menudo se descretizan o separan en "bins" para su análisis.
Supongamos que tiene datos sobre un grupo de personas en un estudio, y desea agruparlos
en cubos discretos de edad:

In [54]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [55]:
bins = [18, 25, 35, 60, 100]
age_categories = pd.cut(ages, bins)
age_categories

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

El objeto que devuelve pandas es un objeto categórico especial. La salida que ve
describe los contenedores calculados por `pandas.cut`. Cada contenedor se identifica
mediante un tipo de valor de intervalo especial (único para pandas) que contiene el
límite y superior de cada contenedor:

In [56]:
age_categories.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [57]:
age_categories.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

Tenga en cuenta que `pd.value_counts(categories)` son los recuentos de basura para el
resultado de `pandas.cut`.

En la representación de cadena de un intervalo, un paréntesis significa que el lado
es abierto (exclusivo), mientras que el soporte cuadrado significa que es cerrado
(inclusive). Puede cambiar qué lado está cerrado al pasar `right=False`:

In [58]:
pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

Puede anular el etiquetado de bin predeterminado basado en intervalos pasando una lista
o matriz a la opción `labels`:

In [59]:
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senio"]
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senio', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senio']

Si pasa un número entero de contenedores a `pandas.cut` en lugar de bordes de contenedor
explícitos, calculará contenedores de igual longitud en función de los valores mínimo
y máximo en los datos. Considere el caso de algunos datos distribuidos uniformemente
cortados en cuartos:

In [60]:
data = np.random.uniform(size=20)
pd.cut(data, 4, precision=2)

[(0.0015, 0.24], (0.48, 0.71], (0.0015, 0.24], (0.48, 0.71], (0.0015, 0.24], ..., (0.48, 0.71], (0.71, 0.95], (0.48, 0.71], (0.0015, 0.24], (0.71, 0.95]]
Length: 20
Categories (4, interval[float64, right]): [(0.0015, 0.24] < (0.24, 0.48] < (0.48, 0.71] < (0.71, 0.95]]

La opción `precision=2` limita la precisión decimal a dos dígitos.

Una función estrechamente relacionada, `pandas.qcut`, contiene los datos basados en
cuantiles de muetras. Dependiendo de la distribución de los datos. Desde `pd.qcut`
utiliza cuantiles de muestra en su lugar, obtendrá contenedores de aproximadamente el
mismo tamaño:

In [61]:
data = np.random.standard_normal(1000)
quartiles = pd.qcut(data, 4, precision=2)
quartiles

[(0.018, 0.7], (0.018, 0.7], (0.7, 3.44], (0.7, 3.44], (0.7, 3.44], ..., (0.7, 3.44], (-0.66, 0.018], (0.7, 3.44], (0.018, 0.7], (0.018, 0.7]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.86, -0.66] < (-0.66, 0.018] < (0.018, 0.7] < (0.7, 3.44]]

In [62]:
pd.value_counts(quartiles)

(-2.86, -0.66]    250
(-0.66, 0.018]    250
(0.018, 0.7]      250
(0.7, 3.44]       250
dtype: int64

Similar a `pandas.cut`, puede pasar sus propios cuantiles (números entre 0 y 1):

In [63]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]).value_counts()

(-2.854, -1.261]    100
(-1.261, 0.0184]    400
(0.0184, 1.314]     400
(1.314, 3.44]       100
dtype: int64

### Detectar y filtrar outliers
Filtrar y transformar valores atípicos es en gran medida una cuestion de aplicar
operaciones de matriz:

In [64]:
data = pd.DataFrame(np.random.standard_normal((100, 4)))
data.describe()

,0,1,2,3
count,100.000000,100.000000,100.000000,100.000000
mean,-0.123960,-0.171141,-0.008600,-0.160098
std,0.953513,0.867705,1.113767,0.950749
min,-2.153270,-2.859757,-3.154859,-2.598513
25%,-0.699240,-0.770114,-0.714229,-0.817798
50%,-0.110546,-0.197564,0.041700,-0.223598
75%,0.535570,0.373901,0.645010,0.411155
max,1.915113,2.256245,2.782615,2.484949


Supongamos que desea encontrar valores en una de las columnas que excede 3 en valor
absoluto:

In [65]:
col = data[2]
col[col.abs() > 3]

62   -3.154859
Name: 2, dtype: float64

Seleccionar todas las filas que tengan un valor superior a 3, puede usar `any` en
un DataFrame booleano:

In [66]:
data[(data.abs() > 3).any(axis="columns")]

,0,1,2,3
62,0.735802,0.272707,-3.154859,0.205444


Los paréntesis alrededor de `data.abs() > 3` son necesarios para llamar a `any` sobre
la operación de comparación.

Los valores se pueden establecer en función de estos críterios. Aquí hay valores de
código para limitar fuera del intervalo 3 a 3:

In [67]:
data[data.abs() >3] = np.sign(data) * 3

In [68]:
data.describe()

,0,1,2,3
count,100.000000,100.000000,100.000000,100.000000
mean,-0.123960,-0.171141,-0.007051,-0.160098
std,0.953513,0.867705,1.109448,0.950749
min,-2.153270,-2.859757,-3.000000,-2.598513
25%,-0.699240,-0.770114,-0.714229,-0.817798
50%,-0.110546,-0.197564,0.041700,-0.223598
75%,0.535570,0.373901,0.645010,0.411155
max,1.915113,2.256245,2.782615,2.484949


La declaración `np.sign(data)` produce valores de 1-1 en función de si los valores en
`data` son positivos o negativos:

In [69]:
np.sign(data).head()

,0,1,2,3
0,1.0,1.0,-1.0,1.0
1,-1.0,-1.0,-1.0,1.0
2,1.0,1.0,-1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0
4,-1.0,1.0,-1.0,-1.0


### Permutación y muestreo aleatorio
Permutar (reordenar aleatoriamente) una Serie o filas en un DataFrame es posible usando
la función `numpy.random.permutation`. Llamando `permutation` con la longitud del eje
que desea permutar produce una matriz de enteros que indica el nuevo orden:

In [70]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))
df

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [71]:
sampler = np.random.permutation(5)
sampler

array([3, 2, 1, 0, 4])

In [72]:
df.take(sampler)

,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
2,14,15,16,17,18,19,20
1,7,8,9,10,11,12,13
0,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34


In [73]:
df.iloc[sampler]

,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
2,14,15,16,17,18,19,20
1,7,8,9,10,11,12,13
0,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34


In [74]:
# seleccionar una permutación por columnas take con axis="columns"
column_sampler = np.random.permutation(7)
column_sampler

array([5, 6, 0, 2, 4, 1, 3])

In [75]:
df.take(column_sampler, axis="columns")

,5,6,0,2,4,1,3
0,5,6,0,2,4,1,3
1,12,13,7,9,11,8,10
2,19,20,14,16,18,15,17
3,26,27,21,23,25,22,24
4,33,34,28,30,32,29,31


Para seleccionar un subconjunto aleatorio sin reemplazo, puede usar el método
`sample` en Series y DataFrame:

In [76]:
df.sample(n=3)

,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
0,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27


Para generar una muestra sin reemplazo (permitir opciones repetidas):

In [77]:
choices = pd.Series([5, 7, -1, 6, 4])
choices.sample(n=10, replace=True)

0    5
3    6
4    4
0    5
4    4
3    6
2   -1
0    5
3    6
3    6
dtype: int64

### Indicador de computación/variables ficticias
Otro tipo de transformación para aplicaciones de modelado estadístico o aprendizaje
automático es convertir una variable categórica en un _dummy_ o indicador de matriz.
Si una columna en un DataFrame tiene `k` valores distintos, derivaría una matriz o
DataFrame con `k` columnas que contiene todos los 1's o 0's. Pandas tiene una función
`pandas.get_dummies` para hacer esto, aunque también podría idear uno usted mismo.

In [78]:
df = pd.DataFrame({"key": list("bbacab"),
                  "data1": range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [79]:
pd.get_dummies(df["key"], dtype=float)

,a,b,c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


En algunos casos, es posible que desee agregar un prefijo a las columnas en el
indicador DataFrame, que luego se puede fusionar con los otros datos.

In [80]:
dummies = pd.get_dummies(df["key"], dtype=float)
df_with_dummy = df[["data1"]].join(dummies)
df_with_dummy

,data1,a,b,c
0,0,0.0,1.0,0.0
1,1,0.0,1.0,0.0
2,2,1.0,0.0,0.0
3,3,0.0,0.0,1.0
4,4,1.0,0.0,0.0
5,5,0.0,1.0,0.0


Si una fila en un DataFrame pertenece a múltiples categorías, tenemos que usar un
enfoque diferente para crear las variables ficticias.

In [83]:
mnames = ["movie_id", "title", "genres"]

movies = pd.read_table("movies.dat", sep="::",
                      header=None, names=mnames, engine="python")
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Pandas ha implementado un método especial de la serie `str.get_dummies` que maneja
este escenario de membresía de grupos múltiples codificada como una cadena delimitada:

In [84]:
dummies = movies["genres"].str.get_dummies("|")
dummies.iloc[:10, :6]

,Action,Adventure,Animation,Children's,Comedy,Crime
0,0,0,1,1,1,0
1,0,1,0,1,0,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0
5,1,0,0,0,0,1
6,0,0,0,0,1,0
7,0,1,0,1,0,0
8,1,0,0,0,0,0
9,1,1,0,0,0,0


Entonces, como antes, puede combinar esto con `movies` mientras agrega un `"genre_"`
a los nombres de columnas en el DataFrame `dummies` con el método `add_prefix`:

In [85]:
movies_windic = movies.join(dummies.add_prefix("Genre_"))
movies_windic

,movie_id,title,genres,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Crime,Genre_Documentary,...,Genre_Fantasy,Genre_Film-Noir,Genre_Horror,Genre_Musical,Genre_Mystery,Genre_Romance,Genre_Sci-Fi,Genre_Thriller,Genre_War,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,Meet the Parents (2000),Comedy,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3879,3949,Requiem for a Dream (2000),Drama,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3880,3950,Tigerland (2000),Drama,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3881,3951,Two Family House (2000),Drama,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Una receta útil para aplicaciones estadísticas es combinar `pandas.get_dummies` con una
función de descretización como `pandas.cut`:

In [86]:
np.random.seed(12345)
values = np.random.uniform(size=10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [88]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


### Extensión tipo de datos